# 依赖

In [216]:
import requests
import json

In [217]:
def save_to_md(file_path, title, original_essay, improved_essay, scores):
    with open(file_path, 'w', encoding='utf-8') as md_file:
        # 写入评分标准
        md_file.write("## Evaluation Criteria\n\n")
        for aspect, score in scores.items():
            md_file.write(f"- **{aspect.capitalize()}:** {score} points\n")
        md_file.write("\n")

        # 写入原始文章
        md_file.write("## Original Essay\n\n")
        md_file.write(original_essay)
        md_file.write("\n\n---\n\n")

        # 写入改进后的范文
        md_file.write("## Improved Essay\n\n")
        md_file.write(improved_essay)
        md_file.write("\n\n---\n\n")

        # 比较改进前后的不同之处
        md_file.write("## Changes\n\n")
        compare_essays(original_essay, improved_essay, md_file)

def compare_essays(original, improved, md_file):
    original_lines = original.split('\n')
    improved_lines = improved.split('\n')

    for i in range(min(len(original_lines), len(improved_lines))):
        if original_lines[i] != improved_lines[i]:
            md_file.write(f"### Line {i + 1}\n")
            md_file.write(f"- Original: {original_lines[i]}\n")
            md_file.write(f"- Improved: {improved_lines[i]}\n\n")

## 读取 API 密钥

In [218]:

with open("key.txt", 'r', encoding='utf-8') as f:
    keys = [i.strip() for i in f.readlines()]


# 定义 ChatAnywhere API 的 URL 和请求头


In [219]:
with open("key.txt", 'r', encoding='utf-8') as f:
    api_key = f.read().strip()
chatanywhere_url = "https://api.chatanywhere.com.cn/v1/chat/completions"
headers = {
    'Authorization': f'Bearer {api_key}',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
}

# 读取文章

In [220]:

def get_essay():
    with open("ielts_essay.txt", 'r', encoding='utf-8') as f:
        text = f.read().split("\n\n")
    title = text[0].strip() + "\n"
    passage = text[1].strip()
    return title, passage

## 聊天机器人交互

In [221]:
def ask_chatanywhere(query):
    payload = json.dumps({
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "user",
                "content": query
            }
        ]
    })

    response = requests.post(chatanywhere_url, headers=headers, data=payload)
    response_json = response.json()
    return response_json['choices'][0]['message']['content']


# 清理文本

In [222]:
def clean(text):
    text = text.replace('<br>', "\n")
    return text

# 整体评估

# 执行整体评估

In [223]:
def overall_assess():
    short_names = {'ta': 'Task Achievement', 'cc': 'Coherence and Cohesion', 'lr': 'Lexical Resource', 'gra': 'Grammatical Range and Accuracy'}
    overall_assessments = []
    prompts = load_prompt(task=2)
    prefix = "Here is the task description:\n"
    
    infix = "Here is the essay for evaluation:\n"

    title, passage = get_essay()
    done = []
    while True:
        for prompt in prompts:
            if prompt[0] in done:
                continue
            query = prompt[1] + "\n" + prefix + title + "\n" + infix + passage
            try:
                response = requests.post("https://api.chatanywhere.com.cn/v1/chat/completions", json={
                    "data": [
                        [[query, None]],
                        keys,
                    ]
                }, headers={'Accept': 'application/json'}).json()

                improved_essay = response["data"][0][-1][1]
                overall_assessments.append(clean(improved_essay))
                done.append(prompt[0])
            except Exception as e:
                print(f"Error processing {prompt[0]}: {str(e)}")
                continue
            print(len(done))
        if len(done) == 4:
            break

    return title, passage, done, overall_assessments, improved_essay


##  在Jupyter中展示结果


In [224]:
print("## Task Description\n" + title + "\n" + "## Essay:\n" + passage.replace("\n", "\n\n") + "\n\n---\n\n")
for short_name, feedback in zip(done, overall_assessments):
    print("## " + short_names[short_name] + "\n" + feedback + "\n\n")

## Task Description
The debate over the role of punishment in teaching children right from wrong has been a contentious issue. While some argue that punishment is a necessary tool for instilling moral values, I firmly believe that a more effective approach lies in positive reinforcement and constructive guidance.

## Essay:
From a young age, children should be taught about the consequences of their actions. However, it is important to note that their understanding of these consequences is largely dependent on their developmental stage. For very young children, punishments may not be fully understood and can instead instill fear or confusion. Instead, positive reinforcement of good behavior and gentle discouragement of negative actions can be more effective at this stage.

---


## Task Achievement
{"id":"chatcmpl-8JwggWU78L7k2KDbg1QIDqxDf6OlL","object":"chat.completion","created":1699764518,"model":"gpt-3.5-turbo-1106","choices":[{"index":0,"message":{"role":"assistant","content":"The 

# 或：保存到markdown文件

In [225]:
save_to_md("ielts_feedback.md", title, passage, overall_assessments[0], {"Task Achievement": 7, "Coherence and Cohesion": 6, "Lexical Resource": 7, "Grammatical Range and Accuracy": 7})
